In [ ]:
#| hide
from agora_l402.core import *

# agora-l402

> agora-l402 exposes agora API as an MCP server and toolset for AI agents

**See Agora MCP in action**:

 Check out this [example conversation with Claude](https://claude.ai/share/4e26c031-07df-430f-9e78-8168f6db94da) where Agora and Fewsats MCPs work together to let users search for products and complete purchases.

# ![Agora and Fewsats integration with Claude](agora-fewsats-claude-chat.png)

## Overview

Agora L402 is an SDK for integrating with the Agora E-commerce API, enabling AI assistants to use the API either as tools or through MCP.

It works with the [Fewsats library](https://github.com/Fewsats/fewsats-python) to allow any product to be purchased through the Fewsats payment system.

The project provides a Python library exposed as tools for AI agents and an MCP server for seamless integration with AI assistants like Claude.



## MCP Server Integration

One of the most powerful features of Agora L402 is its Machine Communication Protocol (MCP) server, which enables AI assistants to interact with e-commerce functionality directly.

The repository includes a ready-to-use MCP server in the `fastmcp` directory. This allows AI assistants like Claude to:

1. Search for products with filtering and sorting
2. View detailed product information
3. Create shopping carts and add items
4. Generate payment intents

For detailed setup instructions, see the [FastMCP README](./fastmcp/README.md).

### Quick Setup

1. Navigate to FastMCP folder `cd fastmcp`
2. Copy `.env.example` to `.env` and fill in the values.
3. Install FastMCP server with `fastmcp install main.py -f .env`

### Payment Flow with Fewsats

When a payment is required, the Agora MCP server creates a payment intent; however, this intent must be fulfilled by the [Fewsats MCP server](https://github.com/Fewsats/fewsats-python). This separation of concerns allows for a more secure and modular approach to e-commerce transactions:

1. Agora MCP: Product search and discovery
2. Fewsats MCP: Payment processing and fulfillment

To see this workflow in action, check out this [conversation with Claude](https://claude.ai/share/4e26c031-07df-430f-9e78-8168f6db94da) where both MCPs are used together to create a complete shopping experience.


## Python SDK

The API is also exposed as a python SDK. Below you can see how to search and generate payment intents:

### Installation

```bash
pip install agora-l402
```

In [ ]:
agora = Agora()

# Search for products with Agora
search_results = agora.search_trial(query="summer dress")
product = search_results.json()['Products'][0]
product

{'name': 'Summer Dress',
 'storeName': 'Jenni Kayne',
 'brand': 'Jenni Kayne',
 '_id': '6669a49ce5ae459b041cedbf',
 'slug': 'summer-dress-ink-stripe-1237cfdf-f7de-4a7f-88cf-a24b977495c1-1718199452733',
 'price': 445,
 'isVerified': False,
 'isBoosted': False,
 'source': 'shopify',
 'images': ['https://cdn.shopify.com/s/files/1/0565/1234/8333/products/summer-dress-ink-stripe-01_5f8d8e35-7b04-4d3e-ac1e-f7fecfba809e.jpg?v=1677108275',
  'https://cdn.shopify.com/s/files/1/0565/1234/8333/products/summer-dress-ink-stripe-5.jpg?v=1677602057',
  'https://cdn.shopify.com/s/files/1/0565/1234/8333/products/summer-dress-ink-stripe-03.jpg?v=1677602057',
  'https://cdn.shopify.com/s/files/1/0565/1234/8333/products/summer-dress-ink-stripe-04.jpg?v=1677602057'],
 'url': 'https://jennikayne.com/products/summer-dress-ink-stripe',
 'agoraScore': 100,
 'priceHistory': [{'price': 445,
   'date': '2024-06-12T13:44:27.912Z',
   '_id': '671466656ca0a1804aaa99bf'}],
 'discountVal': 0}

In [ ]:

# Generate payment intent (handled by Agora MCP)
payment_intent = agora.create_payment_intent(title=product['name'], amount=product['price'], currency='USD', offer_id=product['_id'], description=product['name']).json()
payment_intent

{'offers': [{'offer_id': '6669a49ce5ae459b041cedbf',
   'amount': 445,
   'currency': 'USD',
   'description': 'Summer Dress',
   'title': 'Summer Dress',
   'payment_methods': ['lightning', 'credit_card'],
   'type': 'one-off'}],
 'payment_context_token': '39016b6b-46e9-44fa-910c-0f06e2fc1c85',
 'payment_request_url': 'https://api.fewsats.com/v0/l402/payment-request',
 'version': '0.2.2'}

## AI Agents
 
The SDK is exposed as a list of tools that AI agents can use directly without MCP. The example below shows showcases this using [Claudette library](https://claudette.answer.ai/core.html), a very convenient wrapper for Claude.

In [ ]:
from agora_l402.core import Agora

agora = Agora()
for tool in agora.as_tools():
    print(tool.__name__)

search_trial
get_product_detail
create_cart
add_to_cart
create_order
track_order
refresh_token
create_payment_intent


In [ ]:
from claudette import Chat, models

model = models[1]; model

system_prompt = 'You are a helpful assistant that has access to a domain purchase API.'
chat = Chat(model, sp=system_prompt, tools=agora.as_tools())

chat.run('Find me the most expensive glasses you can.')